In [ ]:
import numpy as np
import math
import pysam
import json

In [ ]:
#given long sequence of base pairs, and k for the length of the k-mer
#output a dictionary that will keep track of the counts of each k-mer seen, using their numeric value
def sequenceToNumber(sequence, k):
    basePairs = {'A' : 0, 'a' : 0, 'C': 1, 'c': 1, 'G': 2, 'g': 2, 'T': 3, 't': 3}
    
    referenceArray = {}
    
    
    kmerToNumber = 0
    kmer = sequence[0: k]
    firstBasePair = 0

    for idx, letter in enumerate(kmer):
        kmerToNumber += basePairs[letter] * (4 ** (k - idx - 1))
        
    if kmerToNumber in referenceArray:
        referenceArray[kmerToNumber] += 1
    else:
        referenceArray[kmerToNumber] = 1
        

    for i in range(k, len(sequence)):
        firstBasePair = basePairs[sequence[i - k]] * 4 ** (k - 1)
        # print("firstBasePair: ", firstBasePair)
        #subtract the first base pair's value
        kmerToNumber -= firstBasePair
        kmerToNumber *= 4
        kmerToNumber += basePairs[sequence[i]]
        
        #save the value of the new first base pair in new k-mer to subtract later

        # print("kmerToNumber: ", kmerToNumber)
        if kmerToNumber in referenceArray:
            referenceArray[kmerToNumber] += 1
        else:
            referenceArray[kmerToNumber] = 1
    
    return referenceArray

In [ ]:
def insertIntoGenome(genome, viralSequence, locations):
    locations.sort()
    finalLocs = []
    count = 0
    for l in locations:
        loc = l + count * len(viralSequence)
        finalLocs.append(loc)
        genome = genome[0:loc] + viralSequence + genome[loc:]
        count += 1
    f = open('chr11.fa', 'w+')
    f.write('>chr11\n')
    writeChunks = [genome[i: i+50] for i in range(0, len(genome), 50)]
    for w in writeChunks:
        f.write(w + "\n")
    return genome, finalLocs

In [ ]:
def createRefArray(sequence, k, fileName):
    referenceArray = {}
    
    for i in range(0, len(sequence)-k):
        string = ""
        if type(sequence) is list:
            string = "".join(sequence[i: i+k])
        else:
            string = sequence[i: i+k]
            
        if string in referenceArray:
            referenceArray[string] += 1
        else:
            referenceArray[string] = 1

#     with open(fileName, 'w+') as file:
#         file.write(json.dumps(referenceArray))
    return referenceArray

In [ ]:
def flipSequence(sequence):
    output = ''
    for bp in sequence:
        if bp == 'A':
            ouput += 'T'
        elif bp == 'T':
            output += 'A'
        elif bp == 'G':
            output += 'C'
        elif bp == 'C':
            output += 'G'
        elif bp == 'N':
            output += 'N'
    return output[::-1]

In [64]:
def readTestSeq(file, finalLocs, viralString, viralRef, humanRef):
    with open(file) as fp:
        numError = 0
        N = 0.0
        isHybrid = False
        for line in fp:
#             print(line)            
            readNumber = 0
            readDirection = 1
            N += 1.0
            if line.startswith("@chr"):
                isHybrid = False
                line = line.strip()
                readNumber = int(line[-1])
                readEnd = 0
                readHeader = line.split('_')
                readStart = int(readHeader[int(readNumber)])
                if int(readHeader[1]) > int(readHeader[2]):
                    if readNumber == 1:
                        readDirection = -1
                        readEnd = readStart - 150
                        temp = readStart
                        readStart = readEnd
                        readEnd = temp
                    else:
                        readDireciton = 1
                        readEnd = readStart + 150
                else:
                    if readNumber == 1:
                        readDirection = 1
                        readEnd = readStart + 150
                    else:
                        readDirection = -1
                        readEnd = readStart - 150
                        temp = readStart
                        readStart = readEnd
                        readEnd = temp
                for loc in finalLocs:
                    if (readStart < loc <= readEnd) or (readStart <= loc + len(viralString) < readEnd):
                        print("hybridRead found")
                        isHybrid = True
                        break
                
            if line[0] in validBasePairs:
                sequence = line
                if readDirection < 0:
                    sequence = flipSequence(line)
                    
                count = 0.0
                predictHybrid = False
                ref = createRefArray(sequence, 15, "150bpRead")
#                 print(ref.keys())
                for kmer in list(ref.keys()):
                    
                    if kmer.lower() in viralRef and kmer.upper() not in humanRef:
                        print(kmer.upper())
                        print("just viral")
                        count += 1.0
                    if kmer.lower() in viralRef and kmer.upper() in humanRef:
                        print(kmer.lower())
                        print("in both")
                        count += 0.5
                if count >= 135.0*(1.0/8.0):
                    predictHybrid = True
#                     print("Hybrid Read")
                    
                if predictHybrid != isHybrid:
                    print("count: " + str(count) + "\tpredictHybrid: " + str(predictHybrid) + "\tisHybrid: " + str(isHybrid))
                    numError += 1
                    
#         print('errorRate: ' + str(numError/N))
        return numError, N

In [ ]:
def readJson(file):
    dictionary = {}
    with open(file, 'r') as fp:
        dictionary = json.load(fp)
        
    return dictionary

In [ ]:
validBasePairs = ['A', 'a', 'C', 'c', 'G', 'g', 'T', 't']

In [ ]:
filepath = 'hpv68'
viralSequence = None
viralString = ""
with open(filepath) as fp:
    for line in fp:
        line = line.strip()
        viralString += line.upper()
#         viralSequence = sequenceToNumber(line, 15)

In [ ]:
genomeFilepath = "../projectSequences/HumanGenome/chr11.fa"
genome = ""
humanStringSequence = []
humanSequence = None
with open(genomeFilepath) as fp:
    for line in fp:
        line = line.strip()
        if not line.startswith('>chr11'):
            for char in line:
                if char in validBasePairs:
                    humanStringSequence.append(char.upper())
                genome += char.upper()
#     humanSequence = sequenceToNumber(humanStringSequence, 15)

# print(humanSequence)

In [ ]:
g, finalLocs = insertIntoGenome(genome, viralString, [6000000, 600050])

In [57]:
print(str(len(viralString)))

7822


In [ ]:
viralRef = None
with open('hpv68') as fp:
    for line in fp:
        viralRef = createRefArray(line, 15, "viralRefArray")

In [ ]:
# humanRef = createRefArray(humanStringSequence, 15, "humanRefArray1")

In [ ]:
# print(len(humanRef))

In [ ]:
readResult = readJson("humanRefArray")

In [59]:
print("GTTAGTCACTGGAAT".lower() in readResult)

False


In [ ]:
humanRef = readResult

In [ ]:
print(finalLocs)

In [66]:
print('GAAGTGCCACAAACA' in humanRef)

False


In [65]:
numError1, N1 = readTestSeq("read.bwa.read1.fastq", finalLocs, viralString, viralRef, humanRef)

atttaaaagtgataa
in both
tgtgcatgtatgtgt
in both
tggggaggatgggga
in both
ggggaggatggggaa
in both
aacatttcaaaaact
in both
TGAACAACAAAAAAC
just viral
ATGTGGATTTAAAGG
just viral
tgtgtatgcatgtat
in both
ttaaattttatgcta
in both
gtgtgtgtatgtgtg
in both
gtgtgtgtatgtgtg
in both
aataaaactgctttt
in both
gtgtgtgtgtatgtg
in both
tgtgtgtgtatgtgt
in both
tgtgtgtgtgtatgt
in both
gtgtgtgtgtatgtg
in both
tgtgtgtgtatgtgt
in both
gtgtgtgtatgtgtg
in both
ccttccttatttttt
in both
cttccttattttttt
in both
tgtgcatgtgtgtgt
in both
gtgcatgtgtgtgtg
in both
tgcatgtgtgtgtgt
in both
gcatgtgtgtgtgta
in both
catgtgtgtgtgtat
in both
tatatgtatgaacaa
in both
atatgtatgaacaac
in both
tgtgtgtgtgtatgt
in both
gtgtgtgtgtatgtg
in both
tgtgtgtgtatgtgt
in both
gtgtgtgtatgtgtg
in both
ttatttttttacaga
in both
CTACATTAGAAACCA
just viral
tacattagaaaccat
in both
attcctcttggtact
in both
gtgtgtgtatgtgtg
in both
tgtgcatgtgtgtgt
in both
gtgcatgtgtgtgtg
in both
tgtgtgtgtgtatgt
in both
gtgtgtgtgtatgtg
in both
tgtgtgtgtatgtgt
in both
gtgtgtg

tgtgcatgtgtgtgt
in both
gtgcatgtgtgtgtg
in both
tgcatgtgtgtgtgt
in both
tttttattaaaaaaa
in both
catgtatgtgtatgt
in both
catgtattactatgc
in both
ACCTTGAGTTACCCA
just viral
CCTTGAGTTACCCAC
just viral
gcatttaaaaggtga
in both
catttaaaaggtgac
in both
ATACATAATTTGTAG
just viral
acaggagaaaatatt
in both
TGTTAATAAAGTATG
just viral
GTTAATAAAGTATGT
just viral
tgtgtgtgtgtatgt
in both
gtgtgtgtgtatgtg
in both
tgtgtgtgtatgtgt
in both
tttaaatttcctaat
in both
TTAAATTTCCTAATG
just viral
aatgtcatatataca
in both
tccatacaatgaaat
in both
tgtgtgtgtatgtgt
in both
aggatggggaaaatg
in both
GGATGGGGAAAATGA
just viral
aatttactttaaaaa
in both
atttactttaaaaac
in both
acagaatttaaaaaa
in both
tgtgcatgtgtgtgt
in both


KeyboardInterrupt: 

In [ ]:
numError2, N2 = readTestSeq("read.bwa.read2.fastq", finalLocs, viralString, viralRef, humanRef)

In [ ]:
print(str(1.0 * (numError1 + numError2) /(N1 + N2)))